In [1]:
# Mount Drive (run & follow auth)
from google.colab import drive
drive.mount('/content/drive')

# Set base project path on your Drive (change if different)
DRIVE_ROOT = "/content/drive/MyDrive/Aftershoot_AI_Challenge"   # <--- change if your folder name is different
!ls -la "$DRIVE_ROOT"


Mounted at /content/drive
total 5
drwx------ 2 root root 4096 Nov  4 14:55 dataset
-rw------- 1 root root  781 Nov  4 17:05 Untitled0.ipynb


In [2]:
# Run in a code cell with ! for shell commands
# (Colab usually has PyTorch preinstalled; these ensure versions and add torchvision)
!pip install -q --upgrade pip
!pip install -q torch torchvision pandas scikit-learn pillow tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.8 MB/s eta 0:00:00


In [3]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


Torch version: 2.8.0+cu126
CUDA available: True
Device name: Tesla T4


device(type='cuda')

In [4]:
import os

# Update paths if your Drive folder structure differs
DATA_ROOT = os.path.join(DRIVE_ROOT, "dataset")
TRAIN_IMG_DIR = os.path.join(DATA_ROOT, "Train", "images")
VAL_IMG_DIR = os.path.join(DATA_ROOT, "Validation", "images")
TRAIN_CSV = os.path.join(DATA_ROOT, "Train", "sliders.csv")
VAL_CSV = os.path.join(DATA_ROOT, "Validation", "sliders_input.csv")
SUBMIT_DIR = os.path.join(DRIVE_ROOT, "submissions")
os.makedirs(SUBMIT_DIR, exist_ok=True)

print("Train images dir:", TRAIN_IMG_DIR)
print("Val images dir:", VAL_IMG_DIR)
print("Train CSV:", TRAIN_CSV)
print("Val CSV:", VAL_CSV)
print("Submit dir:", SUBMIT_DIR)


Train images dir: /content/drive/MyDrive/Aftershoot_AI_Challenge/dataset/Train/images
Val images dir: /content/drive/MyDrive/Aftershoot_AI_Challenge/dataset/Validation/images
Train CSV: /content/drive/MyDrive/Aftershoot_AI_Challenge/dataset/Train/sliders.csv
Val CSV: /content/drive/MyDrive/Aftershoot_AI_Challenge/dataset/Validation/sliders_input.csv
Submit dir: /content/drive/MyDrive/Aftershoot_AI_Challenge/submissions


In [5]:
import pandas as pd
train_df = pd.read_csv(TRAIN_CSV)
val_df = pd.read_csv(VAL_CSV)

print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("Train columns:", list(train_df.columns))
print("Val columns:", list(val_df.columns))
train_df.head()


Train shape: (2538, 14)
Val shape: (493, 12)
Train columns: ['copyCreationTime', 'captureTime', 'touchTime', 'id_global', 'grayscale', 'aperture', 'flashFired', 'focalLength', 'isoSpeedRating', 'shutterSpeed', 'Temperature', 'Tint', 'currTemp', 'currTint']
Val columns: ['copyCreationTime', 'captureTime', 'id_global', 'grayscale', 'hasDevelopAdjustmentsEx', 'aperture', 'flashFired', 'focalLength', 'isoSpeedRating', 'shutterSpeed', 'currTemp', 'currTint']


,copyCreationTime,captureTime,touchTime,id_global,grayscale,aperture,flashFired,focalLength,isoSpeedRating,shutterSpeed,Temperature,Tint,currTemp,currTint
0,-63113817600,2024-01-14T16:56:50.67,741426608.1,C68C8010-495C-4427-9F4D-664C2D71EFAD,0,2.970854,1,100.0,1250,7.965784,4150,2,6317,4
1,-63113817600,2023-05-28T20:08:51.87,741426607.1,8EFC0EC0-0936-41CC-81BD-513B35D2CB23,0,7.614710,1,24.0,500,3.000000,4700,4,5767,13
2,-63113817600,2023-06-09T20:54:30.13,741426607.2,4A28220F-024E-4637-80ED-B4533578AFEB,0,3.614710,1,125.0,1000,7.321928,5000,12,5496,6
3,-63113817600,2022-06-10T13:02:12.98,741426606.5,05A76E40-9B2C-40FD-95D4-EF976598640C,0,2.970854,0,40.0,320,7.965784,3150,9,3730,12
4,-63113817600,2023-09-30T17:08:05,741426607.8,B63A179E-232C-4133-BB24-8784B60DECEE,0,2.970854,0,55.0,800,7.643856,3633,4,3661,-6


In [10]:
# ========================================
# 🧹 Step — Preprocessing (fixed for actual columns)
# ========================================

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define numeric columns that exist in both CSVs
numeric_cols = [
    'grayscale', 'aperture', 'flashFired',
    'focalLength', 'isoSpeedRating', 'shutterSpeed',
    'currTemp', 'currTint'
]

# Drop extra columns not needed
drop_cols = ['copyCreationTime', 'captureTime', 'touchTime', 'hasDevelopAdjustmentsEx']
for c in drop_cols:
    if c in train_df.columns:
        train_df.drop(columns=c, inplace=True)
    if c in val_df.columns:
        val_df.drop(columns=c, inplace=True)

# Clean numeric data safely (no warning)
def clean_numeric(df, cols):
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        # safer replacement (works in pandas 3.0+)
        df[c] = df[c].fillna(df[c].mean())
    return df

train_df = clean_numeric(train_df, numeric_cols)
val_df = clean_numeric(val_df, numeric_cols)

# Normalize numeric features
feature_scaler = StandardScaler()
train_df[numeric_cols] = feature_scaler.fit_transform(train_df[numeric_cols])
val_df[numeric_cols] = feature_scaler.transform(val_df[numeric_cols])

# Scale targets (Temperature & Tint)
target_scaler = StandardScaler()
train_df[['Temperature','Tint']] = target_scaler.fit_transform(train_df[['Temperature','Tint']])

print("✅ Preprocessing successful!")
print("Train shape:", train_df.shape)
print("Val shape:", val_df.shape)
print("Numeric cols:", numeric_cols)


✅ Preprocessing successful!
Train shape: (2538, 11)
Val shape: (493, 9)
Numeric cols: ['grayscale', 'aperture', 'flashFired', 'focalLength', 'isoSpeedRating', 'shutterSpeed', 'currTemp', 'currTint']


In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np

class AftershootDataset(Dataset):
    def __init__(self, df, img_dir, numeric_cols, is_train=True, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.numeric_cols = numeric_cols
        self.is_train = is_train
        self.transform = transform or transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor()
        ])
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['id_global']}.tif")
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        metadata = np.array(row[self.numeric_cols].astype(float).values, dtype=np.float32)
        metadata = torch.tensor(metadata, dtype=torch.float32)
        if self.is_train:
            label = np.array([float(row['Temperature']), float(row['Tint'])], dtype=np.float32)
            label = torch.tensor(label, dtype=torch.float32)
            return image, metadata, label
        else:
            return image, metadata, row['id_global']

# Create datasets + loaders
train_dataset = AftershootDataset(train_df, TRAIN_IMG_DIR, numeric_cols, is_train=True)
val_dataset = AftershootDataset(val_df, VAL_IMG_DIR, numeric_cols, is_train=False)

BATCH_SIZE = 32   # T4 can handle larger batches; lower if OOM
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

print("Train batches:", len(train_loader), "Val batches:", len(val_loader))


Train batches: 80 Val batches: 16


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [13]:
import torch.nn as nn
from torchvision import models

class AftershootModel(nn.Module):
    def __init__(self, metadata_dim):
        super().__init__()
        # Pretrained ResNet18
        self.cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.cnn.fc = nn.Identity()   # remove classifier; output 512-d
        # metadata MLP
        self.mlp = nn.Sequential(
            nn.Linear(metadata_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        # fusion and head
        self.fc = nn.Sequential(
            nn.Linear(512 + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)   # Temperature, Tint
        )
    def forward(self, img, meta):
        img_feat = self.cnn(img)
        meta_feat = self.mlp(meta)
        x = torch.cat([img_feat, meta_feat], dim=1)
        out = self.fc(x)
        return out

model = AftershootModel(metadata_dim=len(numeric_cols)).to(device)
print(model)


AftershootModel(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import math

# Criterion, optimizer, scheduler
criterion = nn.L1Loss()   # MAE
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

EPOCHS = 20
best_loss = math.inf
save_path = os.path.join(SUBMIT_DIR, "aftershoot_model_v2.pth")

for epoch in range(EPOCHS):
    model.train()
    running = 0.0
    for imgs, metas, labels in tqdm(train_loader, total=len(train_loader)):
        imgs = imgs.to(device, non_blocking=True)
        metas = metas.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        outs = model(imgs, metas)
        loss = criterion(outs, labels)
        loss.backward()
        optimizer.step()
        running += loss.item()
    scheduler.step()
    train_loss = running / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} — Train MAE: {train_loss:.4f}")
    # Save best
    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model.state_dict(), save_path)
        print("Saved best model ->", save_path)

print("Training finished. Best MAE:", best_loss)


100%|██████████| 80/80 [04:28<00:00,  3.36s/it]


Epoch 1/20 — Train MAE: 0.5490
Saved best model -> /content/drive/MyDrive/Aftershoot_AI_Challenge/submissions/aftershoot_model_v2.pth


100%|██████████| 80/80 [00:14<00:00,  5.39it/s]


Epoch 2/20 — Train MAE: 0.4110
Saved best model -> /content/drive/MyDrive/Aftershoot_AI_Challenge/submissions/aftershoot_model_v2.pth


100%|██████████| 80/80 [00:16<00:00,  4.79it/s]


...

Epoch 18/20 — Train MAE: 0.1441
Saved best model -> /content/drive/MyDrive/Aftershoot_AI_Challenge/submissions/aftershoot_model_v2.pth


100%|██████████| 80/80 [00:16<00:00,  4.81it/s]


Epoch 19/20 — Train MAE: 0.1388
Saved best model -> /content/drive/MyDrive/Aftershoot_AI_Challenge/submissions/aftershoot_model_v2.pth


100%|██████████| 80/80 [00:15<00:00,  5.09it/s]

Epoch 20/20 — Train MAE: 0.1453
Training finished. Best MAE: 0.13878028346225618


In [15]:
# ========================================
# 🧾 Step — Inference (with inverse scaling)
# ========================================

import pandas as pd
from tqdm import tqdm

# Load best trained model
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

predictions = []

with torch.no_grad():
    for imgs, metas, ids in tqdm(val_loader, total=len(val_loader)):
        imgs = imgs.to(device, non_blocking=True)
        metas = metas.to(device, non_blocking=True)

        # Model output
        outs = model(imgs, metas).cpu().numpy()

        # Inverse transform normalized targets
        outs = target_scaler.inverse_transform(outs)

        # Round and store predictions
        for i, id_ in enumerate(ids):
            temp, tint = outs[i]
            predictions.append([str(id_), int(round(temp)), int(round(tint))])

# Save predictions
pred_df = pd.DataFrame(predictions, columns=['id_global', 'Temperature', 'Tint'])
output_path = os.path.join(SUBMIT_DIR, "predictions.csv")
pred_df.to_csv(output_path, index=False)

print("✅ Predictions saved successfully!")
print("📁 File path:", output_path)
print("🔢 Total predictions:", len(pred_df))
print("📊 Columns:", list(pred_df.columns))
print("📍 Preview:")
print(pred_df.head())


100%|██████████| 16/16 [01:19<00:00,  4.94s/it]

✅ Predictions saved successfully!
📁 File path: /content/drive/MyDrive/Aftershoot_AI_Challenge/submissions/predictions.csv
🔢 Total predictions: 493
📊 Columns: ['id_global', 'Temperature', 'Tint']
📍 Preview:
                              id_global  Temperature  Tint
0  EB5BEE31-8D4F-450A-8BDD-27C762C75AA6            0     1
1  DE666E1F-0433-4958-AEC0-9A0CC0F81036            0     1
2  F6A6EA9C-A5C2-4BBA-9812-5CE52B818CB6            0     1
3  BCC39DEF-598C-491A-A3CA-14A249717F36            0     1
4  390ED94E-0066-4822-99B9-8F1568BDFBF5            0     1


In [16]:
# ========================================
# 🧾 FINAL INFERENCE (Corrected & Scaler Fixed)
# ========================================

import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

# Load your best model checkpoint
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

# 🔹 Fit scaler again on original (unscaled) training data
raw_train = pd.read_csv(TRAIN_CSV)
from sklearn.preprocessing import StandardScaler
true_scaler = StandardScaler()
true_scaler.fit(raw_train[['Temperature', 'Tint']])

predictions = []

# 🔹 Generate predictions for validation data
with torch.no_grad():
    for imgs, metas, ids in tqdm(val_loader, total=len(val_loader)):
        imgs = imgs.to(device, non_blocking=True)
        metas = metas.to(device, non_blocking=True)

        # Forward pass
        outs = model(imgs, metas).cpu().numpy()

        # Convert back to original Temperature/Tint scale
        outs = true_scaler.inverse_transform(outs)

        # Collect all predictions
        for i, id_ in enumerate(ids):
            temp, tint = outs[i]
            predictions.append([
                str(id_),
                int(round(float(temp))),
                int(round(float(tint)))
            ])

# 🔹 Save predictions to Drive
pred_df = pd.DataFrame(predictions, columns=['id_global', 'Temperature', 'Tint'])
output_path = os.path.join(SUBMIT_DIR, "predictions.csv")
pred_df.to_csv(output_path, index=False)

# 🔹 Display summary
print("✅ Predictions saved successfully!")
print("📁 File path:", output_path)
print("🔢 Total predictions:", len(pred_df))
print("📊 Columns:", list(pred_df.columns))
print("📍 Preview:")
print(pred_df.head())

# Optional: Check approximate value range
print("\nTemperature Range:", pred_df['Temperature'].min(), "to", pred_df['Temperature'].max())
print("Tint Range:", pred_df['Tint'].min(), "to", pred_df['Tint'].max())


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 16/16 [00:04<00:00,  3.73it/s]

✅ Predictions saved successfully!
📁 File path: /content/drive/MyDrive/Aftershoot_AI_Challenge/submissions/predictions.csv
🔢 Total predictions: 493
📊 Columns: ['id_global', 'Temperature', 'Tint']
📍 Preview:
                              id_global  Temperature  Tint
0  EB5BEE31-8D4F-450A-8BDD-27C762C75AA6         5276    14
1  DE666E1F-0433-4958-AEC0-9A0CC0F81036         5254    17
2  F6A6EA9C-A5C2-4BBA-9812-5CE52B818CB6         5550    14
3  BCC39DEF-598C-491A-A3CA-14A249717F36         5324    17
4  390ED94E-0066-4822-99B9-8F1568BDFBF5         5497    15

Temperature Range: 2089 to 12341
Tint Range: 0 to 27
